In [185]:
import pdoc
import json
import inspect
import bamboost
from docstring_parser import parse


INCLUDE_METHODS = {"__getitem__", "__setitem__", "__len__", "__iter__"}


def parse_docstring(docstring: str):
    # Extract parameters and returns from docstring using regex
    doc = parse(docstring)
    return {
        "description": f"{doc.short_description} {doc.long_description if doc.long_description else ''}".replace(
            "\n", " "
        ),
        "arguments": {
            param.arg_name.split()[0]: param.description for param in doc.params
        },
        "return": doc.returns.description if doc.returns else None,
    }


def document_method(method: pdoc.doc.Function, is_classmethod=False):
    docstring = parse_docstring(method.docstring)
    signature = method.signature

    method_data = {
        "docstring": docstring["description"],
        "signature": signature.__str__(),
        "returns": {
            "annotation": str(signature.return_annotation),
            "description": docstring["return"].replace("\n", " ")
            if docstring["return"]
            else None,
        },
        "arguments": {
            key: {
                "default": str(val.default) if val.default != inspect._empty else None,
                "annotation": str(val.annotation)
                if val.annotation != inspect._empty
                else None,
                "description": docstring["arguments"].get(key).replace("\n", " ")
                if key in docstring["arguments"]
                else None,
            }
            for key, val in signature.parameters.items()
        },
        "source": {
            "code": method.source,
            "lines": method.source_lines,
        },
        "props": {
            "isClassMethod": is_classmethod,
        },
    }

    return method_data


def document_instance_variable(variable: pdoc.doc.Variable):
    return {
        "annotation": variable.annotation_str,
        "description": parse_docstring(variable.docstring)["description"].replace(
            "\n", " "
        )
        if variable.docstring
        else None,
    }


def document_class(cls: pdoc.doc.Class):
    own_members = cls.own_members
    classmethods = {i.name for i in cls.classmethods}
    class_docstring = parse_docstring(cls.docstring)

    result = {
        "name": cls.name,
        "docstring": class_docstring["description"],
        "methods": {},
        "variables": {},
        "inherits_from": {
            i[0]: [
                (j.type, j.name)
                for j in cls.inherited_members[i]
                if not j.name.startswith("_") or j.name in INCLUDE_METHODS
            ]
            for i in cls.inherited_members
            if i[0] != "builtins"
        },
    }

    # Constructor
    constructor = cls.members.get("__init__", None)
    result["constructor"] = {
        "signature": constructor.signature_without_self.__str__(),
        "arguments": {
            key: {
                "default": str(val.default) if val.default != inspect._empty else None,
                "annotation": (
                    str(val.annotation) if val.annotation != inspect._empty else None
                ),
                "description": class_docstring["arguments"].get(key).replace("\n", " ")
                if key in class_docstring["arguments"]
                else None,
            }
            for key, val in constructor.signature.parameters.items()
        },
        "source": {"code": constructor.source, "lines": constructor.source_lines},
    }

    for member in own_members:
        if member.name.startswith("_") and member.name not in INCLUDE_METHODS:
            continue
        if isinstance(member, pdoc.doc.Function):
            result["methods"][member.name] = document_method(
                member, is_classmethod=member.name in classmethods
            )
            continue
        elif isinstance(member, pdoc.doc.Variable):
            result["variables"][member.name] = document_instance_variable(member)
            continue

    return result


def document_module(module: pdoc.doc.Module):
    result = {
        "name": module.fullname,
        "docstring": parse_docstring(module.docstring)["description"].strip(),
        "classes": {cls.name: document_class(cls) for cls in module.classes},
        "functions": {func.name: document_method(func) for func in module.functions},
        "submodules": {
            sub_module.name: document_module(sub_module)
            for sub_module in module.submodules
        },
    }

    return result


clsManager = pdoc.doc.Module(bamboost.manager).classes[0]
res = {
    "Manager": document_class(clsManager),
    "Simulation": document_class(pdoc.doc.Module(bamboost.simulation).classes[0]),
    "SimulationWriter": document_class(
        pdoc.doc.Module(bamboost.simulation_writer).classes[0]
    ),
}

bam = pdoc.doc.Module(bamboost.simulation)
bam.fullname

/home/florez/.local/lib/python3.10/site-packages/pdoc/doc_types.py:147: UserWarning: Error parsing type annotation pd.Dataframe for bamboost.simulation.Simulation.data_info. Import of pandas.Dataframe failed: module 'pandas' has no attribute 'Dataframe'
  warnings.warn(
/home/florez/.local/lib/python3.10/site-packages/pdoc/doc_types.py:147: UserWarning: Error parsing type annotation pd.Dataframe for bamboost.simulation_writer.SimulationWriter.data_info. Import of pandas.Dataframe failed: module 'pandas' has no attribute 'Dataframe'
  warnings.warn(


'bamboost.simulation'

In [186]:
with open(
    "/home/florez/work/code/bamboost-docs/extract-docs/data/source_docs.json", "w"
) as f:
    f.write(json.dumps(document_module(pdoc.doc.Module(bamboost)), indent=2))

In [192]:
import os

source_docs = document_module(pdoc.doc.Module(bamboost))

header_tmpl = """---
title: {title}
---

import {{ RenderClass, RenderModule }} from '@site/src/components/SourceDocumentation'
import {{ TableOfContents }} from '@site/src/components/TOC';
import sourceDoc from '@site/extract-docs/data/source_docs.json';
"""

basepath = "/home/florez/work/code/bamboost-docs/docs/autoDocs"

doc_card_links = """
import DocCardList from '@theme/DocCardList';

<DocCardList />
"""


def create_md_for_module(module: dict) -> None:
    submodules = module["submodules"]

    basename = module["name"].split(".")[-1]
    module_path = "/".join(module["name"].split(".")[1:-1])

    filename = f"{basename}.md"
    card_links = ""

    if len(submodules) > 0:
        for key, submodule in submodules.items():
            create_md_for_module(submodule)

        # Overwrite module_path and filename
        module_path = "/".join(module["name"].split(".")[1:])
        filename = f"index.md"
        card_links = doc_card_links

    module_path = os.path.join(basepath, module_path)
    os.makedirs(module_path, exist_ok=True)
    f = open(os.path.join(module_path, filename), "w")

    header = header_tmpl.format(title=basename)
    f.write(header)
    f.write(f"\n## TOP\n")
    f.write(
        f'\n<RenderModule data={{sourceDoc}} moduleFullName="{module["name"]}" />\n'
    )
    f.write(card_links)

    for cls, obj in module["classes"].items():
        f.write(f"\n## {cls}\n")
        f.write(
            f'\n<RenderClass data={{sourceDoc}} classFullName="{module["name"]}.{cls}" />\n'
        )

    f.write("\n<TableOfContents />\n")
    f.close()


for module in source_docs["submodules"]:
    create_md_for_module(source_docs["submodules"][module])

In [182]:
source_docs["submodules"]["common"]

{'name': 'bamboost.common',
 'docstring': 'None ',
 'classes': {},
 'functions': {},
 'submodules': {'file_handler': {'name': 'bamboost.common.file_handler',
   'docstring': 'None ',
   'classes': {'FileHandler': {'name': 'FileHandler',
     'docstring': 'File handler for an hdf5 file with the purpose of handling opening and closing of the file. We use the concept of composition to include an object of this type in classes which need access to an hdf5 file (such as the hdf5pointer and Simulation.)',
     'methods': {'__getitem__': {'docstring': 'None ',
       'signature': '(self, key) -> Any',
       'returns': {'annotation': 'typing.Any', 'description': None},
       'arguments': {'self': {'default': None,
         'annotation': None,
         'description': None},
        'key': {'default': None, 'annotation': None, 'description': None}},
       'source': {'code': '    @capture_key_error\n    def __getitem__(self, key) -> Any:\n        return self.file_object[key]\n',
        'lines